In [17]:
import psycopg2
import pandas as pd
# from sqlalchemy import create_engine
# from sqlalchemy.orm import Session

''' Для подавления сообщений об ошибках при обработке pandas ответов psycopg2'''
import warnings
warnings.filterwarnings('ignore')

In [18]:
''' Аттрибуты подключения содержатся в отдельном файле для безопасности.
    Импортируются в словарь, который далее используется при подключении
    и запросах к базе.
'''
import creds as c_

CONN_ATTR = {
    'user': c_.USER,
    'password': c_.PASSWORD,
    'host': c_.HOST,
    'port': c_.PORT,
    'dbname': c_.DBNAME
    }

def make_request(query_: str) -> object:
    ''' Делает запросы к БД с определёнными аттрибутами
    Arguments:
        query_ [str] -- Строка SQL-запроса (можно многострочник)
    Returns:
        [object] -- Ответ на запрос
    '''
    with psycopg2.connect(**CONN_ATTR) as conn:
        return pd.read_sql_query(query_, conn)

In [19]:
CONN_STR = f'postgresql+psycopg2:///{c_.USER}:{c_.PASSWORD}@{c_.HOST}:{c_.PORT}/{c_.DBNAME}'

def make_request_sa1(query_):
    with create_engine(CONN_STR) as conn:
        return pd.read_sql_query(query_, conn)

def make_request_sa2(query_):
    ENGINE = create_engine(CONN_STR)
    with ENGINE.connect() as conn:
        output_ = pd.read_sql_query(query_, conn.execute(text(query_)))
        # conn.close()
        return output_
def make_request_sa3(query_):
    conn = create_engine(CONN_STR)
    output_ = pd.read_sql_query(query_, conn)
    # conn.close_all()
    return output_


### Задание 4.1 ###

Написать запрос, который позволит узнать, сколько (`cnt`) вакансий в каждом регионе (`area`).
Выбрать пятёрку лидеров по количеству вакансий:

***Ответ - Москва, Санкт-Петербург, Минск, Новосибирск, Алматы***

In [24]:
query_4_1 = f'''
SELECT
    a.name AS area,
    COUNT(*) AS cnt
FROM public.vacancies AS v
JOIN public.areas AS a ON
    v.area_id = a.id
GROUP BY a.id
ORDER BY cnt DESC
LIMIT 10
'''
display(make_request(query_4_1))

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


### Задание 4.4 ###

Написать запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (`schedule`) и типа трудоустройства (`employment`), используемого в вакансиях. Какая пара находится на втором месте по популярности?

***Ответ - Удалённая работа + Полная занятость***

In [27]:
query_4_4 = f'''
SELECT
    schedule,
    employment,
    COUNT(*) AS cnt
FROM public.vacancies
GROUP BY schedule, employment
ORDER BY cnt DESC
OFFSET 1
LIMIT 1
'''
display(make_request(query_4_4))

,schedule,employment,cnt
0,Удаленная работа,Полная занятость,7802


### Задание 5.1 ###

Написать запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

***Ответ - Яндекс / Газпром нефть***

In [29]:
query_5_1 = f'''
SELECT
    e.name
FROM public.vacancies AS v
JOIN public.employers AS e ON
    v.employer_id = e.id
GROUP BY e.name
ORDER BY COUNT(*) DESC
LIMIT 5
'''
display(make_request(query_5_1))

,name
0,Яндекс
1,Ростелеком
2,Тинькофф
3,СБЕР
4,Газпром нефть


### Задание 5.2 ###

Написать запрос, который для каждого региона выведет количество работодателей и вакансий в нём.

Среди регионов, в которых нет вакансий, найти тот, в котором наибольшее количество работодателей. Вписать его название в поле ниже в том виде, который вернул запрос.

***Ответ - Россия***

In [50]:
query_5_2 = f'''
SELECT
    a.name,
    COUNT(DISTINCT e.id) AS employers_cnt,
    COUNT(DISTINCT v.id) AS vacancies_cnt
FROM public.areas AS a
LEFT JOIN public.employers AS e ON
    a.id = e.area
LEFT JOIN public.vacancies AS v ON
    a.id = v.area_id
GROUP BY a.id
ORDER BY vacancies_cnt, employers_cnt DESC
LIMIT 1
'''
display(make_request(query_5_2))

,name,employers_cnt,vacancies_cnt
0,Россия,410,0


In [55]:
query_5_2_1 = f'''
SELECT
    a.name,
    COUNT(DISTINCT e.id) AS employers_cnt,
    COUNT(DISTINCT v.id) AS vacancies_cnt
FROM public.areas AS a
LEFT JOIN public.employers AS e ON
    a.id = e.area
LEFT JOIN public.vacancies AS v ON
    a.id = v.area_id
GROUP BY a.id
ORDER BY vacancies_cnt, employers_cnt DESC
OFFSET 19
LIMIT 20
'''
display(make_request(query_5_2_1))

,name,employers_cnt,vacancies_cnt
0,Австралия,6,0
1,Кировская область,6,0
2,Швейцария,5,0
3,Ивановская область,5,0
4,Канада,5,0
5,Тульская область,5,0
6,Новосибирская область,5,0
7,Липецкая область,5,0
8,Иркутская область,4,0
9,Пензенская область,4,0
